<a href="https://colab.research.google.com/github/KYOH95/Project_4_Boardgame_App_Ver2/blob/main/project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 구글 드라이브로부터 데이터 불러오기

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
import os

#데이터 세트(캐글): https://www.kaggle.com/datasets/extralime/20000-boardgames-dataset
dir = './drive/MyDrive/Project4'
filename = os.listdir(dir)

csv_path = os.path.join(dir, filename[0])
df = pd.read_csv(csv_path)
df.head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (29,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,type,id,thumbnail,image,primary,alternate,description,yearpublished,minplayers,...,War Game Rank,Customizable Rank,Children's Game Rank,RPG Item Rank,Accessory Rank,Video Game Rank,Amiga Rank,Commodore 64 Rank,Arcade Rank,Atari ST Rank
0,0,boardgame,30549,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,Pandemic,"['EPIZOotic', 'Pandemia', 'Pandemia 10 Anivers...","In Pandemic, several virulent diseases have br...",2008,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,boardgame,822,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,Carcassonne,"['Carcassonne Jubilee Edition', 'Carcassonne: ...",Carcassonne is a tile-placement game in which ...,2000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,boardgame,13,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...,Catan,"['CATAN', 'Catan (Колонизаторы)', 'Catan telep...","In CATAN (formerly The Settlers of Catan), pla...",1995,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.shape

(21631, 56)

##1. 데이터 전처리

####1-1. 불필요한 컬럼(특성) 제거

In [14]:
#make meaningful dataframe with valualble columns from the whole dataset
df = df[["id","thumbnail","image","primary","description","yearpublished","minplayers","maxplayers","playingtime","minplaytime",
    "maxplaytime","minage","boardgamecategory","usersrated","average","Board Game Rank","averageweight"]]
df.shape

(21631, 17)

####1-3. 결측치 제거

In [15]:
df.isnull().sum()

id                     0
thumbnail             15
image                 15
primary                0
description            1
yearpublished          0
minplayers             0
maxplayers             0
playingtime            0
minplaytime            0
maxplaytime            0
minage                 0
boardgamecategory    283
usersrated             0
average                0
Board Game Rank        0
averageweight          0
dtype: int64

In [16]:
#Remove null values
df = df.dropna(axis = 0)
df.shape

(21333, 17)

####1-3. 이상치 제거

In [17]:
# remove games which published year are below 1900 or above 2022
condition = (df["yearpublished"]<1900)|(df["yearpublished"]>2022)
df.drop(df.loc[condition].index, inplace=True)
df.shape

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(21040, 17)

In [18]:
# remove games which minimun players are below 1 or maximum players are lower than minplayers
condition = (df["minplayers"]<=0)|(df["maxplayers"]<df["minplayers"])
df.drop(df.loc[condition].index, inplace=True)
df.shape

(20882, 17)

In [19]:
# remove games which both minimun play time and maximum play time are 0
condition = (df["minplaytime"]==0)&(df["maxplaytime"]==0)
df.drop(df.loc[condition].index, inplace=True)
df.shape

(20358, 17)

In [20]:
# remove games if min_play_time and max_play_time were put in opposite way
condition = (df["maxplaytime"]<df["minplaytime"])
df.drop(df.loc[condition].index, inplace=True)
df.shape

(20216, 17)

In [40]:
# Check games if average below 1
condition = (df["averageweight"]<1)
df.drop(df.loc[condition].index, inplace=True)
df.shape

(19804, 17)

In [45]:
# Check games if minage below 0 or above 200
condition = (df["minage"]<0)|(df["minage"]>200)
df[condition]

# Check games if usersrated below 0
condition = (df["usersrated"]<=0)
df[condition]

# Check games if average below 0
condition = (df["average"]<=0)
df[condition]
df.shape

(19804, 17)

1-4. 중복 데이터 제거

In [60]:
#check any duplicated data set
df.duplicated().sum()

0

##2. 학습 할 "description" 데이터 프레임 전처리

In [66]:
#데이터 평가 많은 순으로 sort
df.sort_values(by=["usersrated","average"],ascending=[False,False])
df.head(3)

,id,thumbnail,image,primary,description,yearpublished,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,usersrated,average,Board Game Rank,averageweight
0,30549,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,Pandemic,"In Pandemic, several virulent diseases have br...",2008,2,4,45,45,45,8,['Medical'],109006,7.58896,106,2.4063
1,822,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,Carcassonne,Carcassonne is a tile-placement game in which ...,2000,2,5,45,30,45,7,"['City Building', 'Medieval', 'Territory Build...",108776,7.41837,191,1.9057
2,13,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...,Catan,"In CATAN (formerly The Settlers of Catan), pla...",1995,3,4,120,60,120,10,"['Economic', 'Negotiation']",108064,7.13598,429,2.3130


In [67]:
#new data set for DLM
df_description = df[["id","primary","description"]]
df_description.head(3)

,id,primary,description
0,30549,Pandemic,"In Pandemic, several virulent diseases have br..."
1,822,Carcassonne,Carcassonne is a tile-placement game in which ...
2,13,Catan,"In CATAN (formerly The Settlers of Catan), pla..."


In [68]:
df_description = df_description[0:1000]
df_description.shape

(1000, 3)

####2-1. 텍스트 기반으로 토큰에 넣기

In [ ]:
df_

####2-2. 토큰 정제(Regex, Lower)

####2-3. 불용어(Stop words)

####2-4. 표제어 추출(Stemming)